In [1]:
# seq2seq language model with rnn
# achtung: character-based!

In [2]:
# tutorial
# https://www.tensorflow.org/tutorials/sequences/text_generation

In [3]:
# basis caparthy on rnns
# http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [4]:
# Note: Enable GPU acceleration to execute this notebook faster. 
# In Colab: Runtime > Change runtime type > Hardware acclerator > GPU. 
# If running locally make sure TensorFlow version >= 1.11. # => ok, is 1.13 in Mai 2019

In [5]:
! conda env list

# conda environments:
#
base                  *  /anaconda3
db2_louvre               /anaconda3/envs/db2_louvre
flaschenpost             /anaconda3/envs/flaschenpost
hackweek19               /anaconda3/envs/hackweek19
mapsy                    /anaconda3/envs/mapsy
reinforcement-learning     /anaconda3/envs/reinforcement-learning
relevance_score          /anaconda3/envs/relevance_score
tensorflow_text          /anaconda3/envs/tensorflow_text
time-series-prediction     /anaconda3/envs/time-series-prediction
twitter                  /anaconda3/envs/twitter



In [6]:
import tensorflow as tf

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [7]:
print(tf.__version__)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [8]:
from __future__ import absolute_import, division, print_function
import numpy as np
import os
import time

In [9]:
tf.enable_eager_execution()

# 1. read input data 

In [10]:
#path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# @param: domain
domain = 'hitler_mein_kampf'
path_to_file = os.path.join('originaltexte', domain + '.txt')

In [11]:
path_to_file

'originaltexte/hitler_mein_kampf.txt'

In [12]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'+ part of text: \n{text[1234:1280]}')
print(f'\n+ length of text: {len(text)}, which corresponds to approx {round(len(text)/1500)} pages')
# 1 din-a-4 Seite ca 1500 Zeichen => der shakespeare korpus umfasst ca 700 Seiten

+ part of text: 
de große Bewegung auf dieser 
Erde ihr Wachsen

+ length of text: 1569104, which corresponds to approx 1046 pages


# 2. create mapping char => int

In [13]:
vocab = sorted(set(text))
print(f'length of vocab: {len(vocab)}')
print(f'vocab: {vocab}')

length of vocab: 85
vocab: ['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ä', 'Ö', 'Ü', 'ß', 'ä', 'ö', 'ü', '„']


In [14]:
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [15]:
char2idx['a']

51

In [16]:
for i, _ in zip(char2idx, range(len(idx2char))):
    print(f'{repr(i)} -> {char2idx[i]}',end='   ') #

'\n' -> 0   ' ' -> 1   '!' -> 2   '"' -> 3   "'" -> 4   '(' -> 5   ')' -> 6   '*' -> 7   ',' -> 8   '-' -> 9   '.' -> 10   '/' -> 11   '0' -> 12   '1' -> 13   '2' -> 14   '3' -> 15   '4' -> 16   '5' -> 17   '6' -> 18   '7' -> 19   '8' -> 20   '9' -> 21   ':' -> 22   ';' -> 23   '?' -> 24   'A' -> 25   'B' -> 26   'C' -> 27   'D' -> 28   'E' -> 29   'F' -> 30   'G' -> 31   'H' -> 32   'I' -> 33   'J' -> 34   'K' -> 35   'L' -> 36   'M' -> 37   'N' -> 38   'O' -> 39   'P' -> 40   'Q' -> 41   'R' -> 42   'S' -> 43   'T' -> 44   'U' -> 45   'V' -> 46   'W' -> 47   'X' -> 48   'Y' -> 49   'Z' -> 50   'a' -> 51   'b' -> 52   'c' -> 53   'd' -> 54   'e' -> 55   'f' -> 56   'g' -> 57   'h' -> 58   'i' -> 59   'j' -> 60   'k' -> 61   'l' -> 62   'm' -> 63   'n' -> 64   'o' -> 65   'p' -> 66   'q' -> 67   'r' -> 68   's' -> 69   't' -> 70   'u' -> 71   'v' -> 72   'w' -> 73   'x' -> 74   'y' -> 75   'z' -> 76   'Ä' -> 77   'Ö' -> 78   'Ü' -> 79   'ß' -> 80   'ä' -> 81   'ö' -> 82   'ü' -> 83   '

In [17]:
print(f'This is an example of the encoding: {text[128:256]} ---> {text_as_int[:13]}')

This is an example of the encoding: gshaft zu Landsberg am Lech anzutreten. 

Damit bot sich mir nach Jahren ununterbrochener Arbeit 
zum ersten Male die Möglichkei ---> [46 65 68 73 65 68 70  1  0  0 25 63  1]


# 3. create Training batches

In [18]:
# sequence prediction 
# e.g. input = Hello, targ_output = ello

In [19]:
# The maximum length sentence we want for a single input in characters
# TODO erhöhen, sollte am Ende z.B. ca 100 sein
# @param sequence length
seq_length = 256
examples_per_epoch = len(text)//seq_length
examples_per_epoch

6129

In [20]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(15):
    print(''.join(idx2char[i.numpy()]), end = ' ')

Instructions for updating:
Colocations handled automatically by placer.
V o r w o r t   
 
 A m   1 . 

In [21]:
text_as_int

array([46, 65, 68, ...,  0,  0,  0])

In [22]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(1):
    print(str(len(item)))
    
    for element in item.numpy()[:13]:
        print(str(idx2char[element]), end = ' ')
sequences

257
V o r w o r t   
 
 A m   

<DatasetV1Adapter shapes: (257,), types: tf.int64>

In [23]:
def sequence_item_to_text(item):
    return(''.join([str(idx2char[element]) for element in item.numpy()]))

In [24]:
for idx, item in enumerate(sequences.take(3)):
    print(f'\nsequence number {idx}:')
    print(''.join([str(idx2char[element]) for element in item.numpy()]))
    #print(f'test: {sequence_item_to_text(item)}')
#  print(repr(''.join([str(idx2char[item.numpy()][i]) for i in range(len(item))] )))


sequence number 0:
Vorwort 

Am 1. April 1924 hatte ich, auf Grund des Urteils- 
spruches des Münchner Volksgerichts von diesem Tage, 
meine Festungshaft zu Landsberg am Lech anzutreten. 

Damit bot sich mir nach Jahren ununterbrochener Arbeit 
zum ersten Male die Möglichkeit

sequence number 1:
, an ein Werk heran- 
zugehen, das von vielen gefordert und von mir selbst als 
zweckmäßig für die Bewegung empfunden wurde. So habe 
ich mich entschlossen, in zwei Bänden nicht nur die Ziele 
unserer Bewegung klarzulegen, sondern auch ein Bild der 
Entwick

sequence number 2:
lung derselben zu zeichnen. Aus ihr wird mehr zu 
lernen sein als aus jeder rein doktrinären Abhandlung. 
Ich hatte dabei auch die Gelegenheit, eine Darstellung 
meines eigenen Werdens zu geben, soweit dies zum Ver- 
ständnis sowohl des ersten als auch des 


In [25]:
# 'Hello' => 'Hell', 'ello'
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return(input_text, target_text)

In [26]:
# now construct whole datasate
dataset = sequences.map(split_input_target)
dataset.take(1)

<DatasetV1Adapter shapes: ((256,), (256,)), types: (tf.int64, tf.int64)>

In [27]:
for input_text, target_text in dataset.take(1):
    print('\n')
    print(f'\n++ input: {sequence_item_to_text(input_text)}')
    print(f'\n++ output: {sequence_item_to_text(target_text)}')




++ input: Vorwort 

Am 1. April 1924 hatte ich, auf Grund des Urteils- 
spruches des Münchner Volksgerichts von diesem Tage, 
meine Festungshaft zu Landsberg am Lech anzutreten. 

Damit bot sich mir nach Jahren ununterbrochener Arbeit 
zum ersten Male die Möglichkei

++ output: orwort 

Am 1. April 1924 hatte ich, auf Grund des Urteils- 
spruches des Münchner Volksgerichts von diesem Tage, 
meine Festungshaft zu Landsberg am Lech anzutreten. 

Damit bot sich mir nach Jahren ununterbrochener Arbeit 
zum ersten Male die Möglichkeit


In [28]:
for input_example, target_example in dataset.take(1):
    for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
        print("Step {:4d}".format(i))
        print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
        print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 46 ('V')
  expected output: 65 ('o')
Step    1
  input: 65 ('o')
  expected output: 68 ('r')
Step    2
  input: 68 ('r')
  expected output: 73 ('w')
Step    3
  input: 73 ('w')
  expected output: 65 ('o')
Step    4
  input: 65 ('o')
  expected output: 68 ('r')


# split into training batches

In [29]:
# remember
# seq_length = 100
# examples_per_epoch = len(text)//seq_length
# eine epoche bestetht aus ca 11T sequenzen, die in 174 batches von 64 durchgejagt werden

#@param batch_size
batch_size = 64

# size for in_memory buffer where data is shuffeled
BATCH_BUFFER = 10000

steps_per_epoch = examples_per_epoch // batch_size
print('steps per epoch : {:2d}'.format(steps_per_epoch))

steps per epoch : 95


In [30]:
print(seq_length)
print(examples_per_epoch)

256
6129


In [31]:
dataset = dataset.shuffle(BATCH_BUFFER).batch(batch_size, drop_remainder = True)

# ich habe also batches je 64 sequenzen a 100 positionen
dataset

<DatasetV1Adapter shapes: ((64, 256), (64, 256)), types: (tf.int64, tf.int64)>

In [32]:
dataset.take(1)

<DatasetV1Adapter shapes: ((64, 256), (64, 256)), types: (tf.int64, tf.int64)>

# build the model

In [33]:
# length of vocab in chars
vocab_size = len(idx2char)

# the embedding dimension
# @param embedding_dim, rnn_units, EPOCHS
embedding_dim = 512

# the number of rnn units
rnn_units = 1024

#number of epochs
EPOCHS = 32

In [34]:
if tf.test.is_gpu_available():
    print('yes, gpu')
    rnn = tf.keras.layers.CuDNNGRU
else:
    print('no gpu')
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation = 'sigmoid'
    )

no gpu


In [35]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    # TODO insert working GRU / RNN layer
    # this does not work, will result in Tensor's shape (128, 128, 1024) is not compatible with supplied shape (128, 1024)
    #tf.keras.layers.GRU
    rnn(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    # tf.keras.layers.GRU(rnn_units, recurrent_activation = 'sigmoid', return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    # tf.keras.layers.GRU(rnn_units),

    tf.keras.layers.Dense(vocab_size)])
    return model

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

In [37]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           43520     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          4721664   
_________________________________________________________________
dense (Dense)                (64, None, 85)            87125     
Total params: 4,852,309
Trainable params: 4,852,309
Non-trainable params: 0
_________________________________________________________________


# apply model

In [38]:
dataset.take(1)

<DatasetV1Adapter shapes: ((64, 256), (64, 256)), types: (tf.int64, tf.int64)>

In [39]:
for input_example_batch, target_example_batch in dataset.take(1): 
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 256)
(64, 256, 85) # (batch_size, sequence_length, vocab_size)


In [40]:
example_batch_predictions[0].shape

TensorShape([Dimension(256), Dimension(85)])

In [41]:
# pro tip: sample instead of argmax!
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy()

In [42]:
len(sampled_indices)

256

# interpret prediction

In [43]:
input = ''.join(idx2char[input_example_batch[0]])
print(f'++ Input: \n {input}\n')

true_sequence = ''.join(idx2char[target_example_batch[0]])
print(f'++ True Output: \n {true_sequence}\n')

predicted_text = ''.join([idx2char[i] for i in sampled_indices])
print(f'++ predicted Output: \n{predicted_text}\›')

++ Input: 
 überlegen sei, 
daß das deutsche 28-Zentimeter-Rohr dem britischen 30,5- 
Zentimeter-Rohr an Schußleistung gar nicht nachstehe!! 

Gerade deshalb aber wäre es Pflicht gewesen, nun eben- 
falls zum 30,5-Zentimeter-Geschütz überzugehen, da das 
Ziel nicht di

++ True Output: 
 berlegen sei, 
daß das deutsche 28-Zentimeter-Rohr dem britischen 30,5- 
Zentimeter-Rohr an Schußleistung gar nicht nachstehe!! 

Gerade deshalb aber wäre es Pflicht gewesen, nun eben- 
falls zum 30,5-Zentimeter-Geschütz überzugehen, da das 
Ziel nicht die

++ predicted Output: 
U
e)D;g7EN*-rqci3Äfy?4?3q39/fLx,iRcoo pRatsT'NoylGßs9b6v)öcWR-JKxegaEdLsaIGhyUvTdS6V2JyD.W ö)-WMwuz3f7Ig(„/Üe0mVMnfe)ahXReCydkA:'häÖÖ9ä2/sö*g'6klv'cBmp5f;WGä„MBrc9shcgAx;e)8vfJrk8.cÜjpßzhXüW"19ß"wX jR5(twMB9 /c gu;tWmKNßcCHnU2x.Lb5/)o9pe
TeÖx5güFj4vm LäÄ2Ö\›


# compute loss

In [44]:
def loss(true_labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(true_labels, logits, from_logits = True)

In [45]:
loss_first_try = loss(target_example_batch, example_batch_predictions)
print(f'loss right now: {loss_first_try}')
print(f'mean scalar loss: {loss_first_try.numpy().mean()}')

loss right now: [[4.4519343 4.4511967 4.44924   ... 4.4581733 4.4417644 4.446073 ]
 [4.4499526 4.4442625 4.4163127 ... 4.4266543 4.441234  4.4364233]
 [4.4371142 4.424239  4.4405313 ... 4.462819  4.457734  4.443111 ]
 ...
 [4.4266396 4.460176  4.4336658 ... 4.4531417 4.4512205 4.4271655]
 [4.4411836 4.4636188 4.4205875 ... 4.4657335 4.417329  4.4312234]
 [4.439581  4.4378843 4.441623  ... 4.44753   4.4568954 4.4170012]]
mean scalar loss: 4.442663192749023


# compile with adam optimizer

In [46]:
model.compile(optimizer = tf.train.AdamOptimizer(),
    loss = loss)

# define checkpoint

In [47]:
#usually epoch_offset = 0
#only > 0 if training is resumed on previously trained weights
checkpoint_dir = os.path.join('./training_checkpoints', domain)

def get_checkpoint_callback(epoch):

    #checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{summed_epoch}')
    checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
    print(f'writing result to {checkpoint_prefix}')
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_prefix,
        save_weights_only = True
        )
    return checkpoint_callback

# train

In [60]:
# for example shakespeare takes 12 mins per epoch. colab gup version only 12 secs

# altes model laden und weiter trainieren
continue_training = True
if continue_training:
    print('will continue training')
    
    # TODO this should be found automatically
    epoch_offset = 13
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=64)
    model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    
else:
    epoch_offset = 0

checkpoint_callback = get_checkpoint_callback(epoch_offset)
history = model.fit(dataset.repeat(), epochs = EPOCHS, steps_per_epoch = steps_per_epoch, callbacks = [checkpoint_callback])

will continue training
writing result to ./training_checkpoints/hitler_mein_kampf/ckpt_{epoch}
Epoch 1/32
94/95 [============================>.] - ETA: 9s - loss: 0.7645 WARNING:tensorflow:From /anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
95/95 [==============================] - 881s 9s/step - loss: 0.7649
Epoch 2/32
95/95 [==============================] - 930s 10s/step - loss: 0.7400
Epoch 3/32
95/95 [==============================] - 999s 11s/step - loss: 0.7152
Epoch 4/32
95/95 [==============================] - 927s 10s/step - loss: 0.6913
Epoch 5/32
95/95 [==============================] - 889s 9s/step - loss: 0.6667
Epoch 6/32
95/95 [==============================] - 884s 9s/s

In [61]:
type(history)

tensorflow.python.keras.callbacks.History

In [62]:
loss_history = history.history['loss']
print(loss_history)

[0.7649159186764767, 0.7400039089353461, 0.7152075792613782, 0.6912617030896638, 0.6666697044121591, 0.6439291634057698, 0.6245683688866465, 0.6061753291832773, 0.5891834240210684, 0.5734181655080695, 0.5594949960708618, 0.5463809223551499, 0.5351045856350347, 0.5239741002258501, 0.5167518819633283, 0.5064329878280037, 0.49667636658016007, 0.48900089922704193, 0.48034161360640276, 0.47168888951602733, 0.46383509384958366, 0.4576116677961851, 0.4514729543736106, 0.4460471583040137, 0.4400315974888049, 0.43407215162327417, 0.42767107486724854, 0.4229529038855904, 0.41688085637594524, 0.4136293684181414, 0.4080237291361156, 0.40285783284588866]


In [63]:
# TODO: warum hängt das hier immer?
# import matplotlib.pyplot as plt
# plt.plot(loss_history)

In [64]:
history.params

{'batch_size': None,
 'epochs': 32,
 'steps': 95,
 'samples': 95,
 'verbose': 1,
 'do_validation': False,
 'metrics': ['loss']}

# load predictions

In [65]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/hitler_mein_kampf/ckpt_32'

In [66]:
# batch size = 1 to keep model simple
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 512)            43520     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           4721664   
_________________________________________________________________
dense_4 (Dense)              (1, None, 85)             87125     
Total params: 4,852,309
Trainable params: 4,852,309
Non-trainable params: 0
_________________________________________________________________


In [68]:
# wofür das hier?
model.build(tf.TensorShape([1, None]))

In [69]:
def generate_text (model, start_string, temperature, out_len):
    
    # model input
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # result list
    generated_text = []
    

    
    # reset model
    model.reset_states()
    
    # predict char per char
    for i in range(out_len):
        
        # generate first prediction (prob Distribution) from start string
        # predicted_id wird dabei jedes mal an input_eval angehängkt
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # now draw sample from probability Distribution
        # temperatrue hat deutlichen einfluss. 1 ist guter wert
        predictions = predictions / temperature #?? Sinn? Scheint ne reine Skalierung zu sein, hat die überhaupt einen Effekt?
        predicted_id = tf.multinomial(predictions, num_samples = 1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        generated_text.append(predicted_id)
    
    return(''.join([idx2char[i] for i in generated_text]))

In [70]:
# the higher the temp the more chaotic the predicted sentence
temperature = 0.4

# how much text should be produced?
out_len =2000

#start string
start_string = 'L'


In [72]:
generated_string = generate_text(model, start_string, temperature, out_len)
with open (os.path.join('text_artifacts', domain + '_' + start_string +'.txt'), 'w') as file:
    file.write(start_string + generated_string)

print(start_string +generated_string )

Lösung des 
österreichisch-serbischen doch mit der ehemaligen Besitzer 
in ihren aller Entschlossenheit eines Saalschutzes konnte 
man auf solche Weise nicht zu einer Entgegnung von vorn- 
herein unterhöhlte das eigene besondere Anlässe angesein- 
dete, dann konnte es einen Weg an sich erfolgt auf 
die Wahltad von Anfang an rein zahlenmäßig oder 
vorhandenen Anhänger zum großen Angriff nicht nur 
verstehen, sondern auch der Krieg nicht unsympathisch. Was 
dann etwa geradezu Freunde zu sich reden, dessen poli- 
tisches Betonen für die Möglichkeit einer Form einer so 
groß. 

Demgabe höchstens noch viele Jahre habe ich ihnen der Zu- 
sammenbruch dieses unmöglichen Vorstellungen und Arbeiterschaft auf 
seine Angriffe zur Geld am deutschen Durchschnitts- und 
Handelsgeführten gegenüber dem vom Tage das deutsche Volk 
an die heiligste Verpflichtung aus. 

Nun war dieses Herz auf die Bewegung die Notwendigkeit 
einer edleren Volkstums zu erhalten. Der 
egoist, furchtbare Brustund des Reiches